# ARQMath Inference

NOTE: If you use Google Colab you need to enable GPU/TPU support first!

---

To run the model on other data, download it first by create a new cell with the following content (adjust URLs):

```bash
!wget -O ./data/my_data.csv http://mydomain.com/my_data.csv
```

Next, adjust the path below:

In [34]:
input_path = './data/qa-pair.csv'

Run the remaining cells:

In [7]:
import pandas as pd
import numpy as np
import os
import torch
import logging
from transformers import TrainingArguments
from transformers.trainer import Trainer, set_seed
from transformers.modeling_auto import AutoModelForSequenceClassification, AutoConfig
from transformers.tokenization_auto import AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from dataset import ARQMathDataset

seed = 0
set_seed(seed)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

In [2]:
!nvidia-smi

Thu Jul  2 21:45:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  On   | 00000000:04:00.0 Off |                  N/A |
| 23%   23C    P8     7W / 250W |   3679MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  On   | 00000000:06:00.0 Off |                  N/A |
| 28%   23C    P8     9W / 250W |  10977MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [3]:
cuda_visible_devices = '0' # check with nvidia-smi
model_name = 'malteos/arqmath-bert-base-cased'
model_cache_dir = './models/'

if os.path.exists(model_cache_dir + model_name):
    logger.info('Using model from cache')
    model_name_or_path = model_cache_dir + model_name
else:
    model_name_or_path = model_name

In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = cuda_visible_devices

In [5]:
config = AutoConfig.from_pretrained(model_name_or_path, num_labels=2) 
model = AutoModelForSequenceClassification.from_config(config)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

2020-07-02 21:45:53 - INFO - filelock -   Lock 140446372121424 acquired on /home/mostendorff/.cache/torch/transformers/c9aee388d6d61cf37b89f0d56a2458f0758d62b6acb6d3cadc5d8f045d2aeb38.52a95ff4a37d7f42e0b049db1ed2acc4c18a0cc0c72d32b8a07b05a92e88e93c.lock
2020-07-02 21:45:53 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/malteos/arqmath-bert-base-cased/config.json not found in cache or force_download set to True, downloading to /home/mostendorff/.cache/torch/transformers/tmp325l5ej9


2020-07-02 21:45:54 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/malteos/arqmath-bert-base-cased/config.json in cache at /home/mostendorff/.cache/torch/transformers/c9aee388d6d61cf37b89f0d56a2458f0758d62b6acb6d3cadc5d8f045d2aeb38.52a95ff4a37d7f42e0b049db1ed2acc4c18a0cc0c72d32b8a07b05a92e88e93c
2020-07-02 21:45:54 - INFO - transformers.file_utils -   creating metadata file for /home/mostendorff/.cache/torch/transformers/c9aee388d6d61cf37b89f0d56a2458f0758d62b6acb6d3cadc5d8f045d2aeb38.52a95ff4a37d7f42e0b049db1ed2acc4c18a0cc0c72d32b8a07b05a92e88e93c
2020-07-02 21:45:54 - INFO - filelock -   Lock 140446372121424 released on /home/mostendorff/.cache/torch/transformers/c9aee388d6d61cf37b89f0d56a2458f0758d62b6acb6d3cadc5d8f045d2aeb38.52a95ff4a37d7f42e0b049db1ed2acc4c18a0cc0c72d32b8a07b05a92e88e93c.lock
2020-07-02 21:45:54 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/b

2020-07-02 21:45:56 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/malteos/arqmath-bert-base-cased/config.json from cache at /home/mostendorff/.cache/torch/transformers/c9aee388d6d61cf37b89f0d56a2458f0758d62b6acb6d3cadc5d8f045d2aeb38.52a95ff4a37d7f42e0b049db1ed2acc4c18a0cc0c72d32b8a07b05a92e88e93c
2020-07-02 21:45:56 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

2020-07-02 21:45:56 - INFO - transformers.tokeniza

2020-07-02 21:45:58 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/malteos/arqmath-bert-base-cased/vocab.txt in cache at /home/mostendorff/.cache/torch/transformers/ed147be021a8a008d531a91dd824d168683b60715b689bf5331918de7d5fd279.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
2020-07-02 21:45:58 - INFO - transformers.file_utils -   creating metadata file for /home/mostendorff/.cache/torch/transformers/ed147be021a8a008d531a91dd824d168683b60715b689bf5331918de7d5fd279.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
2020-07-02 21:45:58 - INFO - filelock -   Lock 140445729604752 released on /home/mostendorff/.cache/torch/transformers/ed147be021a8a008d531a91dd824d168683b60715b689bf5331918de7d5fd279.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1.lock


2020-07-02 21:45:59 - INFO - filelock -   Lock 140445700782544 acquired on /home/mostendorff/.cache/torch/transformers/12ec75fba2ef9cba3f5f1b3c97acfe2d133cfb533099287da9b508a229baeae7.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4.lock
2020-07-02 21:45:59 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/malteos/arqmath-bert-base-cased/special_tokens_map.json not found in cache or force_download set to True, downloading to /home/mostendorff/.cache/torch/transformers/tmpoa63kfv0


2020-07-02 21:45:59 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/malteos/arqmath-bert-base-cased/special_tokens_map.json in cache at /home/mostendorff/.cache/torch/transformers/12ec75fba2ef9cba3f5f1b3c97acfe2d133cfb533099287da9b508a229baeae7.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4
2020-07-02 21:45:59 - INFO - transformers.file_utils -   creating metadata file for /home/mostendorff/.cache/torch/transformers/12ec75fba2ef9cba3f5f1b3c97acfe2d133cfb533099287da9b508a229baeae7.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4
2020-07-02 21:45:59 - INFO - filelock -   Lock 140445700782544 released on /home/mostendorff/.cache/torch/transformers/12ec75fba2ef9cba3f5f1b3c97acfe2d133cfb533099287da9b508a229baeae7.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4.lock


2020-07-02 21:45:59 - INFO - filelock -   Lock 140445700783312 acquired on /home/mostendorff/.cache/torch/transformers/fc9c4dcb89f7b906ea952df0d5f31b97b288332be9226bdcb90e81f989e47600.73a933aa27255ce576c445dcdb8155b6edb6e4c43cceb14b4b81f9e699a818b7.lock
2020-07-02 21:45:59 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/malteos/arqmath-bert-base-cased/tokenizer_config.json not found in cache or force_download set to True, downloading to /home/mostendorff/.cache/torch/transformers/tmp0sgqj2kq


2020-07-02 21:46:00 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/malteos/arqmath-bert-base-cased/tokenizer_config.json in cache at /home/mostendorff/.cache/torch/transformers/fc9c4dcb89f7b906ea952df0d5f31b97b288332be9226bdcb90e81f989e47600.73a933aa27255ce576c445dcdb8155b6edb6e4c43cceb14b4b81f9e699a818b7
2020-07-02 21:46:00 - INFO - transformers.file_utils -   creating metadata file for /home/mostendorff/.cache/torch/transformers/fc9c4dcb89f7b906ea952df0d5f31b97b288332be9226bdcb90e81f989e47600.73a933aa27255ce576c445dcdb8155b6edb6e4c43cceb14b4b81f9e699a818b7
2020-07-02 21:46:00 - INFO - filelock -   Lock 140445700783312 released on /home/mostendorff/.cache/torch/transformers/fc9c4dcb89f7b906ea952df0d5f31b97b288332be9226bdcb90e81f989e47600.73a933aa27255ce576c445dcdb8155b6edb6e4c43cceb14b4b81f9e699a818b7.lock


2020-07-02 21:46:00 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/malteos/arqmath-bert-base-cased/vocab.txt from cache at /home/mostendorff/.cache/torch/transformers/ed147be021a8a008d531a91dd824d168683b60715b689bf5331918de7d5fd279.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
2020-07-02 21:46:00 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/malteos/arqmath-bert-base-cased/added_tokens.json from cache at None
2020-07-02 21:46:00 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/malteos/arqmath-bert-base-cased/special_tokens_map.json from cache at /home/mostendorff/.cache/torch/transformers/12ec75fba2ef9cba3f5f1b3c97acfe2d133cfb533099287da9b508a229baeae7.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4
2020-07-02 21:46:00 - INFO - transformers.tokenizati

In [9]:
training_args = TrainingArguments(
    output_dir='./results',
    per_device_eval_batch_size=16,   # batch size for evaluation
    seed=seed,
)

trainer = Trainer(
    model=model,
    args=training_args,
)

2020-07-02 21:49:25 - INFO - transformers.training_args -   PyTorch: setting up devices
2020-07-02 21:49:28 - INFO - transformers.trainer -   You are instantiating a Trainer but W&B is not installed. To use wandb logging, run `pip install wandb; wandb login` see https://docs.wandb.com/huggingface.


In [15]:
# Load CSV from disk
df = pd.read_csv(input_path, index_col=False)
df['rel'] = df['rel'].astype(int)
df.head()

,qID,aID,q,a,rel
0,112,115,What are gradients and how would I use them?\n...,"The ∇ (pronounced ""del"") is an operator, more ...",1
1,118,125,How would you describe calculus in simple term...,There came a time in mathematics when people e...,1
2,118,148,How would you describe calculus in simple term...,One of the greatest achievements of human civi...,0
3,118,517,How would you describe calculus in simple term...,Calculus is basically a way of calculating rat...,0
4,118,127,How would you describe calculus in simple term...,Calculus is a field which deals with two seemi...,0


In [27]:
df = df.sample(n=500).reset_index()

In [28]:
ds = ARQMathDataset(df, tokenizer, 512)

In [30]:
prediction_output = trainer.predict(ds)

2020-07-02 21:54:48 - INFO - transformers.trainer -   ***** Running Prediction *****
2020-07-02 21:54:48 - INFO - transformers.trainer -     Num examples = 500
2020-07-02 21:54:48 - INFO - transformers.trainer -     Batch size = 16


In [32]:
pred_df = df
pred_df['predicted_label_ids'] = prediction_output.predictions.argmax(-1)
pred_df['predicted_0'] = prediction_output.predictions[:,0]
pred_df['predicted_1'] = prediction_output.predictions[:,1]       
pred_df.head()

,index,qID,aID,q,a,rel,predicted_label_ids,predicted_0,predicted_1
0,64437,1187021,1187044,Relevance and implications of concavity?\n\nJu...,Concavity/convexity is a great source of usefu...,0,0,0.055390,-0.102732
1,125914,2416180,2416226,About using the lagrange multiplier.\n\nToday ...,We will use the quadratic equation approach to...,0,0,0.127301,-0.067571
2,123140,2356000,2356004,Exam 9768 of subject GRE Q1.\nThis is the que...,HINT : let G be a primitive of g(t) = log(t) t...,0,0,0.077603,-0.064002
3,59714,1099141,1099145,"if f(x) if differentiable and continuous, prov...",Hint: apply the Lagrange theorem to the functi...,1,0,0.300498,0.020297
4,7207,127689,1963775,Why does an infinite limit not exist?\n\nI rea...,Note that working in the affinely extended rea...,0,0,0.211263,-0.054389


In [33]:
pred_df.to_csv('./results/inference_predictions.csv')

In [ ]:
# Download (only on Google Colab)
from google.colab import files
files.download('./results/inference_predictions.csv')